In [1]:
import numpy as np #linear algebra
import pandas as pd #data preprocessing
import matplotlib.pyplot as plt #data visualization
import h5py
import PIL

import utils.general_utils as util
from utils.general_utils import populate_breeds, get_imgMatrix_from_id, get_breed_value_from_id, get_filename_from_id
#other built utilities

from data_loader import dataFrameBuilder

#using inception_v3 to classify dog breeds
import tensorflow as tf #import tensroflow



In [2]:
#PARAMETERS
image_size = 500 #all images are size image_size x image_size x 3
batch_size = 10
num_classes = 120

In [3]:
%matplotlib inline

#prepare csv files
train = pd.read_csv("../data/included/labels.csv")
test = pd.read_csv("../data/included/test_id.csv")
BREED_LIST = "../data/preprocessed_data/breed_list.csv"

In [4]:
#prepare the breed list dataframe
labels = populate_breeds(BREED_LIST) #get the list of all dog breeds
labels_np = np.array(labels).reshape(120,1) #labels list reshaped to numpy array

In [ ]:
#x = tf.placeholder(tf.float32, shape=[None, 500,500, 3], name='input_data')

x = tf.keras.layers.Input(shape=(500,500,3), batch_size=batch_size,name='input_data',dtype='float32')

y = tf.placeholder(tf.float32, shape=[None, 120,1], name='correct_labels')
#x=tf.placeholder(tf.float32, shape=[500,500,3],name='input_data')

y_pred = tf.placeholder(tf.float32, shape=[None,120,1], name='predicted_labels')


In [8]:
def train_input_fn(index=0, data_amnt = None):
    input_img_data = dataFrameBuilder(data_amnt,start_index=index)
    #df.shuffle().repeat().batch(batch_size)
    #print(df.sample(1))
    #input_img_data = df.as_matrix(columns=['Image Data'])
    #input_img_data.reshape([500,500,3])
    input_img_data = np.asarray(input_img_data)
    
    return input_img_data

def train_output_fn(index=0,data_amnt = None):
    output_breed_data = dataFrameBuilder(data_amnt,
                                         start_index=index,
                                      ret_input=False,
                                      ret_output=True)
    #return df.as_matrix(columns=['Breed'])
    output_breed_data = np.asarray(output_breed_data)
    return output_breed_data

In [ ]:
inception_v3 = tf.keras.applications.InceptionV3(include_top=False,
                                                input_tensor=x,
                                                classes=120)

#set imagedata to channels_last for best performance


In [ ]:
# add a global spatial average pooling layer
output_layer = inception_v3.output
output_layer = tf.keras.layers.GlobalAveragePooling2D()(output_layer)
# let's add a fully-connected layer
output_layer = tf.keras.layers.Dense(1024, activation='relu')(output_layer)
# and a logistic layer -- let's say we have 200 classes
predictions = tf.keras.layers.Dense(120, activation='softmax')(output_layer)

In [ ]:
# this is the model we will train
model = tf.keras.Model(inputs=inception_v3.input, outputs=predictions)

In [ ]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

In [6]:
def generator(features, labels, batch_size):
 # Create empty arrays to contain batch of features and labels#
 batch_features = np.zeros((batch_size, 500, 500, 3))
 batch_labels = np.zeros((batch_size,120))
 while True:
   for i in range(batch_size):
     # choose random index in features
     index= random.choice(len(features),1)
     batch_features[i] = some_processing(features[index])
     batch_labels[i] = labels[index]
   yield batch_features, batch_labels


In [11]:
tf.reset_default_graph()

with tf.Session() as ss:
    x = tf.keras.layers.Input(shape=(500,500,3), batch_size=batch_size,name='input_data',dtype='float32')

    y = tf.placeholder(tf.float32, shape=[None, 120,1], name='correct_labels')
    #x=tf.placeholder(tf.float32, shape=[500,500,3],name='input_data')

    y_pred = tf.placeholder(tf.float32, shape=[None,120,1], name='predicted_labels')

    
    inception_v3 = tf.keras.applications.InceptionV3(include_top=False,
                                                     weights='imagenet',
                                                    input_tensor=x,
                                                    classes=120)

    #steps for adding a new output layer
    output_layer = inception_v3.output
    output_layer = tf.keras.layers.GlobalAveragePooling2D()(output_layer) #replace the current global avg pool 2d
    output_layer = tf.keras.layers.Dense(1024, activation='relu')(output_layer) 
    predictions = tf.keras.layers.Dense(120, activation='softmax')(output_layer) #120 classes in the new model
    
    model = tf.keras.Model(inputs=inception_v3.input, outputs=predictions)
                                   
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                        optimizer='sgd')
    
    img_data = train_input_fn(data_amnt=batch_size)
    breed_data = train_output_fn(data_amnt=batch_size)
    
    model.fit(x=img_data, y=breed_data, batch_size=batch_size)
    #inception_v3.fit_generator(generator)
    #model.fit_generator(generator(features, labels, batch_size), samples_per_epoch=50, nb_epoch=10)
    #print(img_data[0])
    
    index = 11
    
    img_data = train_input_fn(index=index, data_amnt=batch_size)
    breed_data = train_output_fn(index=index, data_amnt=batch_size)
    model.evaluate(x=img_data,y=breed_data,batch_size=batch_size)
    

Epoch 1/1
10/10 [==============================]10/10 [==============================] - 40s 4s/step - loss: 4.7683

10/10 [==============================]10/10 [==============================] - 10s 1s/step

